## Journey API


### Setup

---


In this notebook, we will use the
[Journey API](https://developer.sbb.ch/apis/journey-service/documentation) which
is SBB's API for public transport journeys. We will use the API to find the next
connections from a given location to a given destination.


In [54]:
import requests
import sys
import configparser
from pprint import pprint
sys.path.insert(0, "..")

Read the configuration of the API such as secrets etc.


In [17]:
config = configparser.ConfigParser()
config.read('../config.ini')

In [ ]:
API_URL = "https://journey-service-int.api.sbb.ch/v3/trips/by-origin-destination"
CLIENT_SECRET = config["Credentials"]["client_secret"]
CLIENT_ID = config["Credentials"]["client_id"]
SCOPE = "c11fa6b1-edab-4554-a43d-8ab71b016325/.default"

### PySBB

---


PySBB is a wrapper around SBB API which allows for easier querying of the API.


In [53]:
import pySBB

connections = pySBB.get_connections("Zürich", "Bern")
for c in connections:
    print(c)

Zürich HB (17:02, Plat. 31) -> Bern (17:58, Plat. 6) | 56min
Zürich HB (17:08, Plat. 18!) -> Bern (18:26, Plat. 6) | 1h 18min
Zürich HB (17:32, Plat. 12) -> Bern (18:28, Plat. 5) | 56min
Zürich HB (17:38, Plat. 15) -> Bern (18:56, Plat. 6) | 1h 33min


### SBB API

---


Below, we will use the SBB API to find the next connections from a given
location to a given destination.


In [55]:
def get_token():
    params = {
        "grant_type": "client_credentials",
        "scope": SCOPE,
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
    }
    token_url = "https://login.microsoftonline.com/2cda5d11-f0ac-46b3-967d-af1b2e1bd01a/oauth2/v2.0/token"
    return requests.post(token_url, data=params).json()["access_token"]

After defining the helper function for getting access token, we can use the for
instance the trips API to get our data:


In [52]:
auth = get_token()
headers = {
    'Authorization': f"Bearer {auth}",
    'accept': 'application/json',
    'Accept-Language': 'en',
    'Content-Type': 'application/json'
}


# Modify the payload according to the API's requirements
payload = {
    "origin": "8501120",
    "destination": "8501037"
}

api_response = requests.post(API_URL, headers=headers, json=payload)
pprint(api_response.json()["trips"][0])

{'duration': 'PT10M',
 'fastTransfer': False,
 'id': 'iwRECgJWMRLdAsKswrZCQUlNwrZ0AQvwPkhLScK2VCRBPTFATz1MYXVzYW5uZUBYPTY2MjkwOTVAWT00NjUxNjc5NUB1PTBATD04NTAxMTIwQGE9MTI4QA05FE1vcmdlcwE3FDQ5Mzk2OQ03CDExMC43AAwwMzdADTcsMjAyMzEyMDIxNjE5HQ1AMjkkSVIgMTUgMjUyNCQkMSQFAfBAwrZLQ8K2I1ZFIzIjQ0YjMTAwI0NBIzAjQ00jMCNTSUNUIzEjQU0jMTE1NTA4MDEjQU0yIzAjUlQjMTUjwrZLQ0MJPSgwI0VSRyMxI0hJTgEMJENLIzUxNjQ5OXwZBwQ1MAUOAQcUMHwwfDg1BRwoODh8MXwwfDQwOTgFFygtMjE0NzQ4MzY0OAFgAFIRYSgxIxqkAQo6ChIKBy0muBIHU1RBVElPThIZMjAyMy0xMi0wMlQxNjoxOTowMCswMTowMCIHUExBTk5FRCgIGTwIMDM3FTwAGjo8AAAyTjwAYAkSGggIEAkaBjAwMDAxMSICSVIqAjE1MgQhSjwqDlBVQkxJQ19KT1VSTkVZ',
 'legs': [{'duration': 'PT10M',
           'id': '0',
           'mode': 'TRAIN',
           'serviceJourney': {'directions': [{'name': 'Genève-Aéroport'}],
                              'id': '1|262417|0|85|2122023|C:B',
                              'notices': [{'advertised': True,
                                           'name': 'WS',
                                 